# Game of Fivers

_Combinatorial Optimization course, FEE CTU in Prague. Created by [Industrial Informatics Department](http://industrialinformatics.fel.cvut.cz)._

## Motivation

Riddle: On a square board of size $(n \times n)$ there lie $n^2$ stones. Every stone has two sides - white and black. In the beginning, all stones have the white side facing upwards.

You may turn the stone (white to black or black to white), but if you do that, all the stones in its 4-neighborhood will be turned as well. You want to reach the state in which all the stones have their black sides facing upwards. 

What is the minimal number of moves you need to do? 

## Input

You are given a positive integer $n \geq 3$, representing the size of the board.


In [22]:
n = 20

## Output

You should find a minimal number of moves that need to be done to reach the final states (all stones black). Also, you should provide the moves (e.g., a list of positions of the stones to be turned over).

## Model

In [23]:
import gurobipy as g

m = g.Model()

X = m.addVars(n, n, vtype=g.GRB.BINARY, name="x")
K = m.addVars(n, n, vtype=g.GRB.INTEGER, name="k")

for i in range(n):
    for j in range(n):
        #corners
        if i == 0 and j == 0:
            m.addConstr(X[i,j] + X[i+1,j] + X[i,j+1] == 2*K[i,j] + 1)
        elif i == n-1 and j == 0:
            m.addConstr(X[i,j] + X[i-1,j] + X[i,j+1] == 2*K[i,j] + 1)
        elif i == 0 and j == n-1:
            m.addConstr(X[i,j] + X[i+1,j] + X[i,j-1] == 2*K[i,j] + 1)
        elif i == n-1 and j == n-1:
            m.addConstr(X[i,j] + X[i-1,j] + X[i,j-1] == 2*K[i,j] + 1)
        #edges
        elif i == 0:
            m.addConstr(X[i,j] + X[i+1,j] + X[i,j+1] + X[i, j-1] == 2*K[i,j] + 1)
        elif i == n-1:
            m.addConstr(X[i,j] + X[i-1,j] + X[i,j+1] + X[i,j-1] == 2*K[i,j] + 1)
        elif j == 0:
            m.addConstr(X[i,j] + X[i+1,j] + X[i,j+1] + X[i-1,j] ==    2*K[i,j] + 1)
        elif j == n-1:
            m.addConstr(X[i,j] + X[i+1,j] + X[i,j-1] + X[i-1,j] == 2*K[i,j] + 1)
        #middle
        else:
            m.addConstr(X[i,j] + X[i+1,j] + X[i,j-1] + X[i-1,j] + X[i,j+1] == 2*K[i,j] + 1)

m.setObjective(X.sum("*", "*"))
m.optimize()

for i in range(n):
    for j in range(n):
        print("x" if X[i,j].X > 0.5 else "-", end="")
    print()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 400 rows, 800 columns and 2320 nonzeros
Model fingerprint: 0xe3ef2a4b
Variable types: 0 continuous, 800 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 21 columns
Presolve time: 0.01s
Presolved: 399 rows, 779 columns, 2249 nonzeros
Variable types: 0 continuous, 779 integer (474 binary)

Root relaxation: objective 9.138907e+01, 999 iterations, 0.05 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   91.38907    0  397          -   91.38907      -     -    0s
     0     0   93.68843    0  428          -   93.68843      -     -    0s
     0

 ##  Visualization

In [24]:
import matplotlib.pyplot as plt
import numpy as np

def visualize(board, n):
    board = np.array(board)
    clicks = np.argwhere(board == 1)
    plt.imshow(board, interpolation='none')
    plt.scatter(clicks[:, 1], clicks[:, 0], c='red')
    plt.gca().set_xticks(np.arange(-0.5, n, 1))
    plt.gca().set_xticklabels([])
    plt.gca().set_yticks(np.arange(-0.5, n, 1))
    plt.gca().set_yticklabels([])
    plt.grid()
    plt.show()
    
visualize(X.X, n)

AttributeError: 'tupledict' object has no attribute 'X'

## Additional exercise

- Try to experiment with the model for different values of parameter $n$.
- See, how far is the model scalable (i.e., is it able to solve the problem for n ~ 10, n ~ 100, or even more in a reasonable time?).